In [ ]:
# imports
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from deepymod.analysis import load_tensorboard
from natsort import natsorted
import os
import pandas as pd

from matplotlib.gridspec import GridSpec


In [ ]:
# settings
sns.set(context='paper', style='white')
col_width = 5.5 # neurips style column width in inches



# Multiple runs

In [ ]:
files = natsorted([file for file in os.listdir('runs') if file.find('SBL')==0])
dfs_sbl = [load_tensorboard(f'runs/{file}') for file in files]

In [ ]:
files = natsorted([file for file in os.listdir('runs') if file.find('pinn')==0])
dfs_pinn = [load_tensorboard(f'runs/{file}') for file in files]

In [ ]:
def mean_std(prop, df_list, log):
    """Returns mean and std and (mean - std, mean + std) of prop for all dataframes in df_list."""
    data = np.stack([prop(df) for df in df_list], axis=-1)
    if log:
        data = np.log10(data)
    mean = np.mean(data, axis=-1)  
    std = np.std(data, axis=-1)  
    return mean, std, (mean-std, mean + std)

In [ ]:
sbl_loss = mean_std(lambda df: df.mse + df.reg, dfs_sbl, log=True)
pinn_loss = mean_std(lambda df: df.mse + df.reg, dfs_pinn, log=True)
x_vals = dfs_pinn[0].index

In [ ]:
plt.plot(x_vals, sbl_loss[0])
plt.fill_between(x_vals, sbl_loss[2][0], sbl_loss[2][1], alpha=0.4)

plt.plot(x_vals, pinn_loss[0])
plt.fill_between(x_vals, pinn_loss[2][0], pinn_loss[2][1], alpha=0.4)

In [ ]:
prior_keys = natsorted(key for key in dfs_sbl[0].keys() if key.find('alpha') == 0)
coeff_keys = natsorted(key for key in dfs_sbl[0].keys() if key.find('coeffs') == 0)
masked_coeff_keys = natsorted(key for key in dfs_sbl[0].keys() if key.find('masked_coeffs') == 0)
bayes_coeff_keys = natsorted(key for key in dfs_sbl[0].keys() if key.find('bayes') == 0)

In [ ]:
sbl_prior = mean_std(lambda df: df[prior_keys], dfs_sbl, log=True)

In [ ]:
plt.plot(x_vals, sbl_prior[0])
for idx in np.arange(sbl_prior[0].shape[-1]):
    plt.fill_between(x_vals, sbl_prior[2][0][:, idx], sbl_prior[2][1][:, idx], alpha=0.4)

In [ ]:
sbl_coeffs = mean_std(lambda df: df[coeff_keys], dfs_sbl, log=False)
pinn_coeffs = mean_std(lambda df: df[coeff_keys], dfs_pinn, log=False)

In [ ]:
plt.plot(x_vals, sbl_coeffs[0])
plt.plot(x_vals, sbl_coeffs[0][:, 2], linewidth=2)
plt.plot(x_vals, sbl_coeffs[0][:, 5], linewidth=2)
for idx in np.arange(sbl_coeffs[0].shape[-1]):
    plt.fill_between(x_vals, sbl_coeffs[2][0][:, idx], sbl_coeffs[2][1][:, idx], alpha=0.4)
    
plt.ylim([-1.2, 1])

In [ ]:
plt.plot(x_vals, pinn_coeffs[0])
plt.plot(x_vals, pinn_coeffs[0][:, 2], linewidth=2)
plt.plot(x_vals, pinn_coeffs[0][:, 5], linewidth=2)
for idx in np.arange(pinn_coeffs[0].shape[-1]):
    plt.fill_between(x_vals, pinn_coeffs[2][0][:, idx], pinn_coeffs[2][1][:, idx], alpha=0.4)
    
plt.ylim([-1.2, 1])

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(0.5 * col_width, 0.7 * col_width))
gs = GridSpec(3, 2, figure=fig)

#Upper left
ax = fig.add_subplot(gs[0, 0])
ax.plot(x_vals, pinn_loss[0], label='PINN')
ax.fill_between(x_vals, pinn_loss[2][0], pinn_loss[2][1], alpha=0.4)

ax.plot(x_vals, sbl_loss[0], label='SBL')
ax.fill_between(x_vals, sbl_loss[2][0], sbl_loss[2][1], alpha=0.4)

#ax.set_ylim([-3.4, -2])
ax.set_title('Loss')
ax.legend()
ax.text(-7000.0, -0.5, 'a', fontdict={'weight':'bold', 'size': 12})
   
    

#Lower left
ax = fig.add_subplot(gs[0, 1])
ax.plot(x_vals, sbl_prior[0])
for idx in np.arange(sbl_prior[0].shape[-1]):
    plt.fill_between(x_vals, sbl_prior[2][0][:, idx], sbl_prior[2][1][:, idx], alpha=0.4)
ax.plot(x_vals, sbl_prior[0][:, 2], linewidth=2)
ax.plot(x_vals, sbl_prior[0][:, 2], linewidth=2)
ax.set_ylim([-0.5, 5.5])
ax.set_title("Prior")
ax.plot(x_vals, sbl_prior[0])
ax.text(-5100.0, 5.3, 'b', fontdict={'weight':'bold', 'size': 12})
   
    
# middle
ax = fig.add_subplot(gs[1, :])
ax.plot(x_vals, pinn_coeffs[0])
ax.plot(x_vals, pinn_coeffs[0][:, 2], linewidth=2)
ax.plot(x_vals, pinn_coeffs[0][:, 5], linewidth=2)
for idx in np.arange(pinn_coeffs[0].shape[-1]):
    ax.fill_between(x_vals, pinn_coeffs[2][0][:, idx], pinn_coeffs[2][1][:, idx], alpha=0.4)
ax.set_xticks([])
ax.set_ylim([-1.2, 0.3])
ax.set_title("Coefficients PINN")
ax.text(-3000.0, 0.25, 'c', fontdict={'weight':'bold', 'size': 12})
    
# right
ax = fig.add_subplot(gs[2, :])
ax.plot(x_vals, sbl_coeffs[0])
ax.plot(x_vals, sbl_coeffs[0][:, 2], linewidth=2)
ax.plot(x_vals, sbl_coeffs[0][:, 5], linewidth=2)
for idx in np.arange(sbl_coeffs[0].shape[-1]):
    ax.fill_between(x_vals, sbl_coeffs[2][0][:, idx], sbl_coeffs[2][1][:, idx], alpha=0.4)
ax.set_xlabel('Epoch', fontdict={'weight':'bold'})
ax.set_ylim([-1.2, 0.3])
ax.set_xticks([0, 5000, 10000])
ax.set_title("Coefficients SBL")
ax.text(-3000.0, 0.25, 'd', fontdict={'weight':'bold', 'size': 12})
plt.savefig('SBL.pdf', bbox_inches='tight')

In [ ]:
run_idx=4
plt.semilogy(dfs_sbl[run_idx][prior_keys].ewm(alpha=0.1).mean())
plt.semilogy(dfs_sbl[run_idx][prior_keys[3]].ewm(alpha=0.1).mean(), lw=3)
plt.semilogy(dfs_sbl[run_idx][prior_keys[6]].ewm(alpha=0.1).mean(), lw=3)

In [ ]:
run_idx=1
plt.plot(dfs_sbl[run_idx][masked_coeff_keys].ewm(alpha=0.1).mean())
plt.plot(dfs_sbl[run_idx][masked_coeff_keys[3]].ewm(alpha=0.1).mean(), lw=3)
plt.plot(dfs_sbl[run_idx][masked_coeff_keys[6]].ewm(alpha=0.1).mean(), lw=3)
plt.ylim([-10, 0.5])

In [ ]:
run_idx=1
plt.plot(dfs_sbl[run_idx][bayes_coeff_keys].ewm(alpha=0.1).mean())
plt.plot(dfs_sbl[run_idx][bayes_coeff_keys[3]].ewm(alpha=0.1).mean(), lw=3)
plt.plot(dfs_sbl[run_idx][bayes_coeff_keys[6]].ewm(alpha=0.1).mean(), lw=3)
plt.ylim([-7, 0.5])

In [ ]:
run_idx=1
plt.plot(dfs_pinn[run_idx][coeff_keys].ewm(alpha=0.1).mean())
plt.plot(dfs_pinn[run_idx][coeff_keys[3]].ewm(alpha=0.1).mean(), lw=3)
plt.plot(dfs_pinn[run_idx][coeff_keys[6]].ewm(alpha=0.1).mean(), lw=3)
plt.ylim([-7, 0.5])

In [ ]:
from jax import random
from modax.data.kdv import doublesoliton
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
key = random.PRNGKey(0)
noise = 0.20

# Making data
x = np.linspace(-10, 10, 100)
t = np.linspace(0.1, 1.0, 20)
t_grid, x_grid = np.meshgrid(t, x, indexing="ij")
u = doublesoliton(x_grid, t_grid, c=[5.0, 2.0], x0=[0.0, -5.0])
u_noisy = u + noise * np.std(u) * random.normal(key, u.shape)

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(0.9 * col_width, 0.3 * col_width))
gs = GridSpec(2, 3, figure=fig)
run_idx=4

frame = 10
ax = fig.add_subplot(gs[0, 0])
ax.contourf(u)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Korteweg - de Vries')
ax.axhline(frame, linestyle='--', color='r')

ax = fig.add_subplot(gs[1, 0])
ax.plot(x, u_noisy[frame, :])
ax.plot(x, u[frame, :], 'k--')
ax.set_yticks([])

ax = fig.add_subplot(gs[:, 1])
ax.plot(dfs_sbl[run_idx][prior_keys].ewm(alpha=0.1).mean())
ax.plot(dfs_sbl[run_idx][prior_keys[3]].ewm(alpha=0.1).mean(), lw=2)
ax.plot(dfs_sbl[run_idx][prior_keys[6]].ewm(alpha=0.1).mean(), lw=2)
ax.set_yscale('log')
ax.axhline(1e4, color='k', linestyle='--')
ax.set_title('Prior')

ax = fig.add_subplot(gs[0, 2])
ax.plot(dfs_sbl[run_idx][masked_coeff_keys].ewm(alpha=0.1).mean())
ax.plot(dfs_sbl[run_idx][masked_coeff_keys[3]].ewm(alpha=0.1).mean(), lw=3)
ax.plot(dfs_sbl[run_idx][masked_coeff_keys[6]].ewm(alpha=0.1).mean(), lw=3)
ax.set_ylim([-6.5, 1.5])
ax.set_xticks([])
ax.set_title('SBL')

ax = fig.add_subplot(gs[1, 2])
ax.plot(dfs_pinn[run_idx][coeff_keys].ewm(alpha=0.1).mean())
ax.plot(dfs_pinn[run_idx][coeff_keys[3]].ewm(alpha=0.1).mean(), lw=3)
ax.plot(dfs_pinn[run_idx][coeff_keys[6]].ewm(alpha=0.1).mean(), lw=3)
ax.set_ylim([-6.5, 3])
ax.set_title('OLS')

plt.savefig('kdv.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
plt.contourf(u)

In [ ]:
u.shape

In [ ]:
frame = 0
plt.plot(x, u[frame, :])
plt.plot(x, u_noisy[frame, :])

In [ ]:
frame = -1
plt.plot(x, u[frame, :])
plt.plot(x, u_noisy[frame, :])